# Visualizing ALDEx2 feature differentials using Qurro

In this example, we use transcriptomic data from [TCGA](https://portal.gdc.cancer.gov/repository). We downloaded 
100 gene expression files from lung squamous cell carcinoma (LUSC) and 49 solid tissue normal expression files. We have pre-processed this data into a feature table for ease of use, but the gdc manifest file is also provided for your convenience as well as the script used to aggregate all the files.

## Requirements:

This notebook requires Qurro, pandas, and biom to be installed for Python. The R packages [ALDEx2](http://bioconductor.org/packages/release/bioc/html/ALDEx2.html) and [dplyr](https://dplyr.tidyverse.org/) are also required for the `run_aldex.R` script.

`NOTE`: This breaks with pandas 1.0.0 due to a biom issue

`TODO`: *insert TCGA citation here*

## 0. Setting up
In this section, we replace the output directory with an empty directory. This just lets us run this notebook multiple times, without any tools complaining about overwriting files.

In [1]:
# Clear the output directory so we can write these files there
!rm -rf output/*
# Since git doesn't keep track of empty directories, create the output/ directory if it doesn't already exist
# (if it does already exist, -p ensures that an error won't be thrown)
!mkdir -p output

## 1. Filtering the feature table

The original feature table has over 60,000 features which is too computationally expensive to process for this example. We will filter this table to use the top 10% of genes by total abundance.

In [2]:
import re
import pandas as pd

In [3]:
feature_table = pd.read_csv(
    "input/TCGA_LUSC_expression_feature_table.tsv",
    sep="\t",
    index_col=0,
)
print(feature_table.shape)
feature_table.head()

(60483, 149)


,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.13,4707,1016,2286,1238,2982,2795,1516,5507,5504,2079,...,1988,1059,1297,1970,2450,1432,1500,1790,3722,2471
ENSG00000000005.5,5,1,1,5,1,0,0,0,2,3,...,1,5,4,0,0,4,3,0,1,0
ENSG00000000419.11,2019,1731,2123,2121,3868,2722,2130,4834,4557,1264,...,1194,1449,1382,747,1741,3167,1454,1569,2477,1826
ENSG00000000457.12,1062,968,1883,655,800,1987,935,898,1014,840,...,449,883,614,678,305,705,917,658,961,470
ENSG00000000460.15,204,202,1923,205,706,2191,956,927,1347,264,...,748,159,180,887,275,181,206,153,581,770


In [4]:
feature_table_filt = feature_table[
    feature_table.sum(axis=1) > feature_table.sum(axis=1).quantile(0.9)
]
print(feature_table_filt.shape)
feature_table_filt.head()

(6049, 149)


,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.13,4707,1016,2286,1238,2982,2795,1516,5507,5504,2079,...,1988,1059,1297,1970,2450,1432,1500,1790,3722,2471
ENSG00000000419.11,2019,1731,2123,2121,3868,2722,2130,4834,4557,1264,...,1194,1449,1382,747,1741,3167,1454,1569,2477,1826
ENSG00000000938.11,5951,6829,1067,11164,1091,492,521,335,1319,4540,...,194,3743,3615,142,1323,5410,7864,1875,805,1591
ENSG00000000971.14,11768,21648,1150,10141,14387,5091,2218,2186,4463,8875,...,1515,12057,5252,534,2250,4630,10018,7015,5265,8355
ENSG00000001036.12,3890,3942,2134,5136,3002,1566,1757,2924,5016,4187,...,2312,3090,4425,2798,1916,3803,4518,3170,3097,3937


We also want to strip everything after the period in the Ensemble IDS. For example, `ENSG00000000003.13` should be converted to `ENSG00000000003`.

In [5]:
feature_table_filt.index = [re.search("ENSG[0-9]*", x).group() for x in feature_table_filt.index]
feature_table_filt.index.name = "feature-id"
feature_table_filt.head()

,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003,4707,1016,2286,1238,2982,2795,1516,5507,5504,2079,...,1988,1059,1297,1970,2450,1432,1500,1790,3722,2471
ENSG00000000419,2019,1731,2123,2121,3868,2722,2130,4834,4557,1264,...,1194,1449,1382,747,1741,3167,1454,1569,2477,1826
ENSG00000000938,5951,6829,1067,11164,1091,492,521,335,1319,4540,...,194,3743,3615,142,1323,5410,7864,1875,805,1591
ENSG00000000971,11768,21648,1150,10141,14387,5091,2218,2186,4463,8875,...,1515,12057,5252,534,2250,4630,10018,7015,5265,8355
ENSG00000001036,3890,3942,2134,5136,3002,1566,1757,2924,5016,4187,...,2312,3090,4425,2798,1916,3803,4518,3170,3097,3937


In [6]:
feature_table_filt.to_csv(
    "output/TCGA_LUSC_expression_feature_table_filt.tsv",
    sep="\t",
    index=True,
)

## 2. Running ALDEx2

ALDEx2 is a tool used for differential abundance between conditions. We have provided a script to run ALDEx2 on the processed feature table. Note that this step can take several minutes.

In [7]:
!Rscript run_aldex.R


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

aldex.clr: generating Monte-Carlo instances and clr values
operating in serial mode
removed rows with sums equal to zero
computing center with all features
data format is OK
dirichlet samples complete
transformation complete
aldex.ttest: doing t-test
running tests for each MC instance:
|------------(25%)----------(50%)----------(75%)----------|
aldex.effect: calculating effect sizes
operating in serial mode
sanity check complete
rab.all  complete
rab.win  complete
rab of samples complete
within sample difference calculated
between group difference calculated
group summaries calculated
effect size calculated
summarizing output
[1] "ALDEx2 results written to output/TCGA_LUSC_aldex_results.tsv"


## 3. Converting feature table from tsv to biom

`<Insert explanation here>`

In [8]:
!biom convert \
    -i output/TCGA_LUSC_expression_feature_table_filt.tsv \
    -o output/TCGA_LUSC_expression_feature_table_filt.biom \
    --table-type="OTU table" \
    --to-hdf5

## 4. Process sample metadata

Qurro expects the metadata file to have the first column labelled as "Sample ID", so we'll edit the sample sheet to comply.

In [9]:
sample_sheet = pd.read_csv("input/gdc_sample_sheet.2020-02-13.tsv", sep="\t")
sample_sheet.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,e4c62f17-d1e8-4543-9b7e-daa2b68306e0,bc5be208-5934-40dd-81df-567599ea2a51.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-6737,TCGA-33-6737-01A,Primary Tumor
1,220a03f7-7ab6-4233-8f65-7ac5decca4b9,60040f95-8414-4956-bd8a-ec461a49207c.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-18-3410,TCGA-18-3410-01A,Primary Tumor
2,8894c42e-ce65-4088-88e3-921ce7165261,950e2ba0-a247-4bd6-8092-f97cc4018a79.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-A4WN,TCGA-33-A4WN-01A,Primary Tumor
3,daa44ce1-1671-46b9-aa48-2f4155f0ee49,a998a5b1-397d-4497-a58c-9b9e1c7f491e.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-56-7579,TCGA-56-7579-01A,Primary Tumor
4,ef056c34-c2b9-47dd-afbf-ed81fc16dc74,840bb854-0669-485e-9d83-c4e1e4f10626.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-34-5236,TCGA-34-5236-01A,Primary Tumor


In [10]:
sample_sheet_new = sample_sheet.set_index("Sample ID", drop=True)
sample_sheet_new.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample Type
Sample ID,,,,,,,
TCGA-33-6737-01A,e4c62f17-d1e8-4543-9b7e-daa2b68306e0,bc5be208-5934-40dd-81df-567599ea2a51.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-6737,Primary Tumor
TCGA-18-3410-01A,220a03f7-7ab6-4233-8f65-7ac5decca4b9,60040f95-8414-4956-bd8a-ec461a49207c.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-18-3410,Primary Tumor
TCGA-33-A4WN-01A,8894c42e-ce65-4088-88e3-921ce7165261,950e2ba0-a247-4bd6-8092-f97cc4018a79.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-A4WN,Primary Tumor
TCGA-56-7579-01A,daa44ce1-1671-46b9-aa48-2f4155f0ee49,a998a5b1-397d-4497-a58c-9b9e1c7f491e.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-56-7579,Primary Tumor
TCGA-34-5236-01A,ef056c34-c2b9-47dd-afbf-ed81fc16dc74,840bb854-0669-485e-9d83-c4e1e4f10626.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-34-5236,Primary Tumor


In [11]:
sample_sheet_new.to_csv("output/sample_metadata.tsv", sep="\t", index=True)

## 5. Mapping Ensembl gene identifiers to HGNC

We might want to know which Ensembl features map to which HUGO features. First, we'll download the `GRCh38` gtf file.

In [12]:
!wget -P output/ ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz

--2020-02-14 16:07:11--  ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz
           => ‘output/Homo_sapiens.GRCh38.99.gtf.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.8
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.8|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-99/gtf/homo_sapiens ... done.
==> SIZE Homo_sapiens.GRCh38.99.gtf.gz ... 46905912
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.99.gtf.gz ... done.
Length: 46905912 (45M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  44.73M  2.62MB/s    in 18s     

2020-02-14 16:07:31 (2.52 MB/s) - ‘output/Homo_sapiens.GRCh38.99.gtf.gz’ saved [46905912]



Next, we'll create a feature metadata file mapping the Ensembl ID to the gene name.

In [13]:
%%bash

echo -e "feature-id\tGene Name" > output/gene_matching.tsv

zgrep "havana\tgene" output/Homo_sapiens.GRCh38.99.gtf.gz | \
awk -F "\t" '{print $9}' | \
awk -F ";" -v OFS="\t" '{print $1, $3}' | \
sed -e "s/gene_[a-z]* //g" | \
tr -d \" >> output/gene_matching.tsv

## 6. Running Qurro

Finally, we'll run Qurro.

In [14]:
%%bash

qurro \
    -r output/TCGA_LUSC_aldex_results.tsv \
    -t output/TCGA_LUSC_expression_feature_table_filt.biom \
    -sm output/sample_metadata.tsv \
    -fm output/gene_matching.tsv \
    -o qurro

Successfully generated a visualization in the folder qurro.


/miniconda3/envs/qurro_revisions/lib/python3.6/site-packages/qurro/_df_utils.py:126: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  table_sdf = pd.SparseDataFrame(table.matrix_data, default_fill_value=0.0)
/miniconda3/envs/qurro_revisions/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
/miniconda3/envs/qurro_revisions/lib/python3.6/site-packages/pandas/core/frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with spa